<a href="https://colab.research.google.com/github/githublior/Monet_cyclegan/blob/main/Training_Notebook_p2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training Notebook - Cylce Gan for picture to Monet 
Student:
Lior shimon 341348498 lior.cbh@gmail.com

In [1]:
 from google.colab import drive
 
drive.mount('/content/drive', force_remount=True)
 
# path = "/content/drive/MyDrive/temp/data"
path = "/content/drive/MyDrive/Colab Notebooks/deep learning Blutman (CGAN)/data"

Mounted at /content/drive


In [2]:
import torch
import random, torch, os, numpy as np
import torch.nn as nn
import torch.nn.functional as F
import copy
from PIL import Image
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import torchvision.models
from torchvision.utils import save_image
import torchvision.transforms
import random
import time
import sys
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from skimage import io




In [3]:
#HYPER Parameters.
 
# import torch
import albumentations as A
# from albumentations.pytorch import ToTensorV2
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
pre_dir  = path + "/301_pic"
pre_m_dir = path + "/aug_monet"
TRAIN_DIR = path +"/data/train"
VAL_DIR = path + "/data/dev"
BATCH_SIZE = 2
LEARNING_RATE_G = 2e-4
LEARNING_RATE_D = 2e-4
LAMBDA_IDENTITY = 5
LAMBDA_CYCLE = 0.5
NUM_WORKERS = 1
WD =  1e-4
OLD =13+13+2
NUM_EPOCHS = 45 - OLD
LOAD_MODEL = True
SAVE_MODEL = True
PRE_G = False 

CHECKPOINT_GEN_U = path+"/genu.pth.tar"
 
CHECKPOINT_GEN_P = path+"/genp.pth.tar"
CHECKPOINT_GEN_M = path+"/genm.pth.tar"
 
CHECKPOINT_CRITIC_P = path+"/criticp.pth.tar"
CHECKPOINT_CRITIC_M = path+"/criticm.pth.tar"
 
 
# data augmentation
transforms = A.Compose(
    [
       
        A.Normalize(mean=[0, 0, 0], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        # ToTensor(),
    ],
    additional_targets={"image0": "image"},
)

In [4]:
# Useful functions.
def save_checkpoint(model, optimizer, filename= path+"/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)
    


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=12):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# this function generate data augmentation from 30 Monet images to 300.
def gen_monet():
  datagen = ImageDataGenerator( 
          featurewise_center = True,
          featurewise_std_normalization = True,
          shear_range = 0.2,
          horizontal_flip = True,
          vertical_flip = True,
          brightness_range = (0.5, 1.5))

  image_directory =  path + '/30_monet' 
  SIZE = 256
  dataset = []
  my_images = os.listdir(image_directory)
  for i, image_name in enumerate(my_images):    
      if (image_name.split('.')[1] == 'jpg'):        
          image = io.imread(image_directory + '/'+image_name)        
          image = Image.fromarray(image, 'RGB')        
          image = image.resize((SIZE,SIZE)) 
          dataset.append(np.array(image))
  x = np.array(dataset)
  i = 0
  for batch in datagen.flow(x, batch_size=1,save_to_dir= path + '/aug_monet' ,save_prefix='dr',save_format='jpg'):    
      i += 1    
      if i > 300:        
        print('here')
        break

# this function select randomly from the 7040 picture dataset (Kaggle): 
#300 picture for preprocess phase, and 300 other picture for the training phase.
def select_some_pic():

  image_directory =  path + '/photo_jpg' #r'C:Users---train/'
  output_dir1 = path + "/300_pic"
  output_dir2 = path + "/301_pic"

  # random.shuffle(image_directory)
  my_images = os.listdir(image_directory)
  for i, filename in enumerate(my_images):   
    img = Image.open(os.path.join(image_directory, filename)) # images are color images
    # print(i)
    if i <300 : img.save(output_dir1+'/'+filename) 
    elif i <600: img.save(output_dir2+'/'+filename) 
    else: break 
      # if (image_name.split('.')[1] == 'jpg'):    
      # print(i)

def Average(lst):
    return sum(lst) / len(lst)

## takes in a module and applies the specified weight initialization
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
        values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
        m.weight.data.normal_(0.0,0.02)#/np.sqrt(y))
        m.bias.data.fill_(0)

In [5]:

# Dataset for training our model 

from IPython.display import Image as Im

class PictureMonetDataset(Dataset):
    def __init__(self, root_monet, root_picture, transform=None):
        self.root_monet = root_monet
        self.root_picture = root_picture
        self.transform = transform

        self.monet_images = os.listdir(root_monet)
        self.picture_images = os.listdir(root_picture)
        self.length_dataset = max(len(self.monet_images), len(self.picture_images))
        self.monet_len = len(self.monet_images)
        self.picture_len = len(self.picture_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        monet_img = self.monet_images[index % self.monet_len]
        picture_img = self.picture_images[index % self.picture_len]

        monet_path = os.path.join(self.root_monet, monet_img)
        picture_path = os.path.join(self.root_picture, picture_img)


        monet_img = np.array(Image.open(monet_path))#.convert("RGB"))
        picture_img = np.array(Image.open(picture_path))#.convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=monet_img, image0=picture_img)
            monet_img = augmentations["image"]
            picture_img = augmentations["image0"]

        monet_img =  np.moveaxis(monet_img, -1,0).astype(float)  # change dimension shape : from  [256,256,3] to [3,256,256] to

        picture_img = np.moveaxis(picture_img, -1,0).astype(float)
      


        return monet_img, picture_img



In [7]:
# Generator Unet from colab + modifications.

#source: https://colab.research.google.com/github/usuyama/pytorch-unet/blob/master/pytorch_unet_resnet18_colab.ipynb#scrollTo=b8EJl0hcC5DH

def convrelu(in_channels, out_channels, kernel, padding):
  return nn.Sequential(
    nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
    nn.InstanceNorm2d(out_channels),
    nn.ReLU(inplace=True),
  )


class ResNetUNet(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.base_model = torchvision.models.resnet18(pretrained=False)
    self.base_layers = list(self.base_model.children())

    self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
    self.layer0_1x1 = convrelu(64, 64, 1, 0)
    self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
    self.layer1_1x1 = convrelu(64, 64, 1, 0)
    self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
    self.layer2_1x1 = convrelu(128, 128, 1, 0)
    self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
    self.layer3_1x1 = convrelu(256, 256, 1, 0)
    self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
    self.layer4_1x1 = convrelu(512, 512, 1, 0)

    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
    self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
    self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
    self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

    self.conv_original_size0 = convrelu(3, 64, 3, 1)
    self.conv_original_size1 = convrelu(64, 64, 3, 1)
    self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

    self.conv_last = nn.Conv2d(64, n_class, 1)

  def forward(self, input):
    x_original = self.conv_original_size0(input)
    x_original = self.conv_original_size1(x_original)

    layer0 = self.layer0(input)
    layer1 = self.layer1(layer0)
    layer2 = self.layer2(layer1)
    layer3 = self.layer3(layer2)
    layer4 = self.layer4(layer3)

    layer4 = self.layer4_1x1(layer4)
    x = self.upsample(layer4)
    layer3 = self.layer3_1x1(layer3)
    x = torch.cat([x, layer3], dim=1)
    x = self.conv_up3(x)

    x = self.upsample(x)
    layer2 = self.layer2_1x1(layer2)
    x = torch.cat([x, layer2], dim=1)
    x = self.conv_up2(x)

    x = self.upsample(x)
    layer1 = self.layer1_1x1(layer1)
    x = torch.cat([x, layer1], dim=1)
    x = self.conv_up1(x)

    x = self.upsample(x)
    layer0 = self.layer0_1x1(layer0)
    x = torch.cat([x, layer0], dim=1)
    x = self.conv_up0(x)

    x = self.upsample(x)
    x = torch.cat([x, x_original], dim=1)
    x = self.conv_original_size2(x)

    out = self.conv_last(x)

    return out

In [8]:
# creating discriminator model

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )
 
    def forward(self, x):
        return self.conv(x)
 
 
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )
 
        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature==features[-1] else 2))
            in_channels = feature
            
        layers.append(nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"))
        self.model = nn.Sequential(*layers)
 
 
    def forward(self, x):
      x = self.initial(x)
      return torch.sigmoid(self.model(x))
 
 
def test():
    x = torch.randn((5, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x)
    
 
    print(preds.shape)
 
 
if __name__ == "__main__":
    test()
 
 

torch.Size([5, 1, 30, 30])


In [9]:
## dataset for preprocessing the U-net (for generators)
class preprocessDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.img = os.listdir(root)
        self.length_dataset = self.len = len(self.img)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        img = self.img[index % self.len]
        path = os.path.join(self.root, img)
        img = np.array(Image.open(path))#.convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=img)
            img = augmentations["image"]

        img =  np.moveaxis(img, -1,0).astype(float)  # change dimension shape : from  [256,256,3] to [3,256,256] to

        return img

In [10]:
# preprocessing the generators

pre_BATCH_SIZE = 5
pre_LEARNING_RATE = 1e-5
pre_NUM_EPOCHS = 1
pre_LOAD_MODEL = False
pre_SAVE_MODEL = True
CHECKPOINT_GEN_U = path+"/genu.pth.tar"

def train_pre(gen, loader, opt_gen, l1, mse, g_scaler,j):

    for (idx,  picture) in enumerate(loader): 
        # print(idx)

        picture = picture.to(DEVICE)
      
        # Train U-net
        with torch.cuda.amp.autocast():
            out_picture = gen(picture)
            picture_loss = mse(picture, out_picture)

        opt_gen.zero_grad()
        g_scaler.scale(picture_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()
        
       
        if idx % 100 == 0:
          if pre_SAVE_MODEL:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN_U)
          save_image(picture, path+f"/pretrain_g/original{idx}.png", normalize=True)          
          save_image(out_picture, path+f"/pretrain_g/output{idx}.png", normalize=True)          


def main():
    model = ResNetUNet(3)
    gen =  model.double().to(DEVICE)

    opt_gen = optim.Adam(list(gen.parameters()), lr=pre_LEARNING_RATE)
    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if pre_LOAD_MODEL:
        load_checkpoint(CHECKPOINT_GEN_U, gen, opt_gen, pre_LEARNING_RATE)
    q = preprocessDataset(path+"/photo_jpg", transform=transforms)
    l = len(q)
    # tr_dataset , val_dataset = torch.utils.data.random_split(q, [.8*l, .2*l], generator=torch.Generator().manual_seed(42))
    # val_loader = DataLoader(val_dataset,batch_size=1)
    tr_loader = DataLoader(q, batch_size=pre_BATCH_SIZE,shuffle=True)
    g_scaler = torch.cuda.amp.GradScaler()

    j=0
    for epoch in range(pre_NUM_EPOCHS):
        j+=1
        start = time.time()
        print('epoch', epoch+1 , 'over',pre_NUM_EPOCHS )
        train_pre(gen, tr_loader, opt_gen, L1, mse, g_scaler,j)
        end = time.time()
        print("time for this epoch:", int((end-start)/60),"min",(end-start)%60,"secs")

        if pre_SAVE_MODEL:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN_U)


if __name__ == "__main__":
    # torch.cuda.empty_cache()
    # seed_everything()
  
    # main()

    print('end')

end


In [11]:
 
# #training cell.
 
def train_fn(disc_P, disc_M, gen_M, gen_P, loader, opt_disc_M,opt_disc_P, opt_gen_M, opt_gen_P, l1, mse,j):#  d_scaler_M,d_scaler_P, g_scaler_M,g_scaler_P, j):
 
   
    dmr = []
    dmrl = []
    dmf = []
    dmfl = []
    dpr = []
    dpf = []
 
    loop = tqdm(loader, leave=True)
 
    disc_P.train()
    disc_M.train()
    gen_M.train()
    gen_P.train()
    
    for idx, (monet, picture) in enumerate(loop):
 
        monet = monet.to(DEVICE)
        picture = picture.to(DEVICE)
 
        # Train Discriminators P and M
 
        fake_monet = gen_M.forward(picture)
        D_M_real = disc_M.forward(monet)
        D_M_fake = disc_M.forward(fake_monet.detach())
        D_M_real_loss = mse(D_M_real, torch.ones_like(D_M_real))
        D_M_fake_loss = mse(D_M_fake, torch.zeros_like(D_M_fake))
        D_M_loss = D_M_real_loss.add(D_M_fake_loss)
 
 
        opt_disc_M.zero_grad()
        D_M_loss.backward()
        opt_disc_M.step()
 
 
        # print('dmf', float(torch.mean(D_M_fake)),'dmf loss',float(D_M_fake_loss) ,'dmr',float(torch.mean(D_M_real)), 'dmr loss',float(D_M_real_loss) )
        dmf.append(float(torch.mean(D_M_fake)))
        dmfl.append(float(D_M_fake_loss))
        dmr.append(float(torch.mean(D_M_real)))
        dmrl.append(float(D_M_real_loss))
    
 
        fake_picture = gen_P.forward(monet)
        D_P_real = disc_P.forward(picture)
        D_P_fake = disc_P.forward(fake_picture.detach())
        D_P_real_loss = mse(D_P_real, torch.ones_like(D_P_real))
        D_P_fake_loss = mse(D_P_fake, torch.zeros_like(D_P_fake))
        D_P_loss = D_P_real_loss.add(D_P_fake_loss)
 
        dpf.append(float(torch.mean(D_P_fake)))
        dpr.append(float(torch.mean(D_P_real)))
 
        opt_disc_P.zero_grad()
        D_P_loss.backward()
        opt_disc_P.step()
#       """   
#         # Train Generators P and M
 
#         # with torch.cuda.amp.autocast():
#  #####
#         # adversarial loss for both generators
#         D_P_fake = disc_P.forward(fake_picture)
#         D_M_fake = disc_M.forward(fake_monet)
        
#         loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))
#         loss_G_M = mse(D_M_fake, torch.ones_like(D_M_fake))
 
 
#         # cycle loss
#         cycle_monet = gen_M.forward(fake_picture)
#         cycle_picture = gen_P.forward(fake_monet)
#         cycle_monet_loss = l1(cycle_monet, monet)
#         cycle_picture_loss = l1(cycle_picture, picture)
# """
  
        D_M_fake = disc_M.forward(fake_monet)
        D_P_fake = disc_P.forward(fake_picture)
 
 
        loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))
        loss_G_M = mse(D_M_fake, torch.ones_like(D_M_fake))
 
 
        # cycle loss
        cycle_monet = gen_M.forward(fake_picture)
        cycle_picture = gen_P.forward(fake_monet)
        cycle_monet_loss = l1(cycle_monet, monet)
        cycle_picture_loss = l1(cycle_picture, picture)
 
        # identity loss (remove these for efficiency if you set lambda_identity=0)
        identity_monet = gen_M.forward(monet)
        identity_picture = gen_P.forward(picture)
        identity_monet_loss = l1(identity_monet, monet)
        identity_picture_loss = l1( identity_picture, picture)
 
        #total cycle loss
        total_cycle_loss = cycle_monet_loss.add(cycle_picture_loss)* LAMBDA_CYCLE
        # add the other loss togethor
        # total_monet_g_loss = loss_G_M.add(identity_monet_loss * LAMBDA_IDENTITY *0.5).add(total_cycle_loss)
        # total_picture_g_loss = loss_G_P.add(identity_picture_loss * LAMBDA_IDENTITY *0.5).add(total_cycle_loss)
 
        idd = (identity_monet_loss + identity_picture_loss ) * LAMBDA_IDENTITY *0.5 
        ttl_g_loss = loss_G_M + loss_G_P + (identity_monet_loss + identity_picture_loss ) * LAMBDA_IDENTITY *0.5  + total_cycle_loss
        
        opt_gen_M.zero_grad()
        opt_gen_P.zero_grad()
        opt_disc_M.zero_grad()
        opt_disc_P.zero_grad()
        
        loss_G_M.backward(retain_graph=True)
        loss_G_P.backward(retain_graph=True)
        total_cycle_loss.backward()
        idd.backward()
 
        # # total_cycle_loss.backward(retain_graph=True)
        # total_monet_g_loss.backward(retain_graph=True)
        # total_picture_g_loss.backward()
        opt_gen_M.step()
        opt_gen_P.step()
        
       
        
 
    save_image(picture, path+f"/sample_in_train/{j+OLD}_picture.png", normalize=True)          
    # save_image(picture * 0.5 + 0.5, path+f"/saved_images/{j+OLD}_picture.png", normalize=True)
    save_image(monet , path+f"/sample_in_train/{j+OLD}_monet.png", normalize=True)
    save_image(fake_picture , path+f"/sample_in_train/{j+OLD}_fp_Gp(m).png", normalize=True)
    save_image(fake_monet, path+f"/sample_in_train/{j+OLD}_fm_Gm(p).png", normalize=True)
    # save_image(cycle_monet , path+f"/sample_in_train/{j+OLD}_cycle_monet.png", normalize=True)
    # save_image(cycle_picture , path+f"/sample_in_train/{j+OLD}_cycle_picture.png", normalize=True)
    save_image(identity_monet , path+f"/sample_in_train/{j+OLD}_identity_monet.png", normalize=True)
    save_image(identity_picture , path+f"/sample_in_train/{j+OLD}_identity_picture.png", normalize=True)
 
    # print(dmf)
    # print(dmr)
    print('for this epoch , dmf',Average(dmf),', dmr',Average(dmr),', dpf',Average(dpf), 'dpr',Average(dpr))#,"loss gen M",Average(loss_g_M), 'total loss disc', Average(loss_d) )
 
 
def main():
    torch.autograd.set_detect_anomaly(True)
    d0  = Discriminator()
    d1 = Discriminator()
    weights_init_normal(d0)    
    weights_init_normal(d1)    
    dm = d0.double().to(DEVICE) # old v : in discr param : in_channels=3
    dp = d1.double().to(DEVICE) # old v : in discr param : in_channels=3
    disc_P = dp
    disc_M = dm
    model_1 = ResNetUNet(3)
    weights_init_normal(model_1)
    gen_M = model_1.double().to(DEVICE)
    model_2 = ResNetUNet(3)
    weights_init_normal(model_2)
    gen_P = model_2.double().to(DEVICE)
 
 
 
    opt_disc_M = optim.Adam(
            list(disc_M.parameters()),
            lr=LEARNING_RATE_D, betas = (0.5, .999))#, weight_decay=WD)
 
    opt_disc_P = optim.Adam(
            list(disc_P.parameters()),
            lr=LEARNING_RATE_D, betas = (0.5, .999))#, weight_decay=WD)
 
 
    opt_gen_P = optim.Adam(
        list(gen_P.parameters()),
        lr=LEARNING_RATE_G,betas = (0.5, .999))#, weight_decay=WD)
    opt_gen_M = optim.Adam(
            list(gen_M.parameters()),
            lr=LEARNING_RATE_G,betas = (0.5, .999))#, weight_decay=WD)
 
    L1 = nn.L1Loss()
    mse = nn.MSELoss()
 
    if PRE_G:
      load_checkpoint(CHECKPOINT_GEN_U, gen_M, opt_gen_M, LEARNING_RATE_G)
      load_checkpoint(CHECKPOINT_GEN_U, gen_P, opt_gen_P, LEARNING_RATE_G)
 
 
    if LOAD_MODEL:
        load_checkpoint(CHECKPOINT_GEN_P, gen_P, opt_gen_P, LEARNING_RATE_G)
        load_checkpoint(CHECKPOINT_GEN_M, gen_M, opt_gen_M, LEARNING_RATE_G )
        load_checkpoint(CHECKPOINT_CRITIC_P, disc_P, opt_disc_P, LEARNING_RATE_D)
        load_checkpoint(CHECKPOINT_CRITIC_M, disc_M, opt_disc_M, LEARNING_RATE_D)
 
 
    dataset = PictureMonetDataset( root_picture= path +"/300_pic", root_monet= path +"/aug_monet", transform=transforms)
    loader = DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,pin_memory=True )
    
 
    j=0
    for epoch in range(NUM_EPOCHS):
        j+=1
        start = time.time()
        print('epoch', epoch+1 , 'over',NUM_EPOCHS )
        train_fn(disc_P, disc_M, gen_M, gen_P, loader, opt_disc_M,opt_disc_P, opt_gen_M,opt_gen_P, L1, mse,j)# d_scaler_M,d_scaler_P, g_scaler_M,g_scaler_P,j)
        end = time.time()
        print("time for this epoch:", int((end-start)/60),"min",(end-start)%60,"secs")
 
        if SAVE_MODEL:
            save_checkpoint(gen_P, opt_gen_P, filename=CHECKPOINT_GEN_P)
            save_checkpoint(gen_M, opt_gen_M, filename=CHECKPOINT_GEN_M)
            save_checkpoint(disc_P, opt_disc_P, filename=CHECKPOINT_CRITIC_P)
            save_checkpoint(disc_M, opt_disc_M, filename=CHECKPOINT_CRITIC_M)
     
if __name__ == "__main__":
    torch.cuda.empty_cache()
    seed_everything()
  
    main()
 
    print('end')

=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint
epoch 1 over 19


  0%|          | 0/151 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 151/151 [14:32<00:00,  5.78s/it]


for this epoch , dmf 0.15080058249863973 , dmr 0.8516147353893185 , dpf 0.24163963424963786 dpr 0.7523945632126902
time for this epoch: 14 min 33.79039168357849 secs
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
epoch 2 over 19


100%|██████████| 151/151 [14:35<00:00,  5.80s/it]


for this epoch , dmf 0.13492286880297222 , dmr 0.8666221024909 , dpf 0.2472318409594928 dpr 0.7514655037926281
time for this epoch: 14 min 35.5405809879303 secs
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
epoch 3 over 19


 48%|████▊     | 72/151 [07:01<07:42,  5.85s/it]


KeyboardInterrupt: ignored